In [1]:
import pandas as pd
import numpy as np


In [2]:
# Cargar los datos
df = pd.read_csv("heart_disease_uci.csv")  # Asegúrate de que el archivo está en el mismo directorio


# **Diccionario de Variables del Dataset**



| Nombre en Dataset | Descripción en Español |
|------------------|--------------------------------------------------|
| `id` | Identificador único del paciente. Lo quitamos del dataset por no ser util en este estudio |
| `age` | Edad del paciente (en años). |
| `sex` | Sexo del paciente (Male = Hombre, Female = Mujer). |
| `dataset` | Origen del dataset (ej. Cleveland, Hungary, etc.). Lo quitamos del dataset por no ser util en este estudio |
| `cp` | Tipo de dolor en el pecho (angina típica, atípica, etc.). |
| `trestbps` | Presión arterial en reposo (mm Hg). |
| `chol` | Colesterol sérico (mg/dL). |
| `fbs` | Glucosa en ayuno (>120 mg/dL: True = Sí, False = No). |
| `restecg` | Resultados del electrocardiograma en reposo. |
| `thalch` | Frecuencia cardíaca máxima alcanzada. |
| `exang` | Angina inducida por ejercicio (True = Sí, False = No). |
| `oldpeak` | Depresión del segmento ST tras el ejercicio. |
| `slope` | Pendiente del segmento ST en el ECG. |
| `ca` | Número de vasos coloreados mediante fluoroscopía. |
| `thal` | Resultado del test de talio (defecto fijo, normal, etc.). |
| `num` | Presencia de enfermedad cardíaca (0 = No, 1-4 = Sí, distintos grados). |

> **Notas:**
> - **ECG (Electrocardiograma en reposo):** Registro de la actividad eléctrica del corazón en reposo.
> - **Segmento ST y prueba de esfuerzo:** Indicadores clave para evaluar isquemia cardíaca.
> - **Número de vasos coloreados (`ca`):** Cuantos vasos sanguíneos fueron visibles en el test de fluoroscopía.
> - **Test de Talio (`thal`):** Evalúa la circulación sanguínea en el corazón mediante imágenes nucleares.



## Limpiamos las columnas que no vamos a usar en este dataset

In [3]:
df.drop(columns=['id', 'dataset'], inplace=True)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,63,Male,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54,Female,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,62,Male,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,55,Male,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,58,Male,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


## Cambiamos los nombres para que sean entendibles

In [4]:
# Diccionario con los nombres en español
columnas_nuevas = {
    'age': 'edad',
    'sex': 'sexo',
    'cp': 'tipo_dolor_pecho',
    'trestbps': 'presion_arterial',
    'chol': 'colesterol',
    'fbs': 'glucosa_ayuno_alta',
    'restecg': 'ecg_reposo',
    'thalch': 'frecuencia_cardiaca_max',
    'exang': 'angina_por_ejercicio',
    'oldpeak': 'depresion_ST',
    'slope': 'pendiente_ST',
    'ca': 'vasos_coloreados',
    'thal': 'prueba_talio',
    'num': 'enfermedad_cardiaca'  # Variable objetivo
}

# Renombrar las columnas
df.rename(columns=columnas_nuevas, inplace=True)

# Mostrar las primeras filas para verificar
df.head()


,edad,sexo,tipo_dolor_pecho,presion_arterial,colesterol,glucosa_ayuno_alta,ecg_reposo,frecuencia_cardiaca_max,angina_por_ejercicio,depresion_ST,pendiente_ST,vasos_coloreados,prueba_talio,enfermedad_cardiaca
0,63,Male,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


## Descripción del método del análisis que vamos a realizar

- en nuestro dataset tenemos varias columnas con valores nulos, pero como vemos que algunos de las columnas con menos datos son determinantes para la predicción vamos a implementar kNN con distancia adaptativa


        Este método permite que kNN ignore las columnas con valores nulos en cada instancia individual, en lugar de eliminar filas o imputar valores.

## Limpieza del dataset
-   Las columnas de **sex**, **fbs** (Glucosa en ayuno (>120 mg/dL: True = Sí, False = No)) y **exang** (Angina inducida por ejercicio (True = Sí, False = No)) las podemos tratar como binarios

In [5]:
# Convertir variables binarias a 0 y 1
df['sexo'] = df['sexo'].map({'Male': 1, 'Female': 0})
df['glucosa_ayuno_alta'] = df['glucosa_ayuno_alta'].map({True: 1, False: 0})
df['angina_por_ejercicio'] = df['angina_por_ejercicio'].map({True: 1, False: 0})

df


,edad,sexo,tipo_dolor_pecho,presion_arterial,colesterol,glucosa_ayuno_alta,ecg_reposo,frecuencia_cardiaca_max,angina_por_ejercicio,depresion_ST,pendiente_ST,vasos_coloreados,prueba_talio,enfermedad_cardiaca
0,63,1,typical angina,145.0,233.0,1.0,lv hypertrophy,150.0,0.0,2.3,downsloping,0.0,fixed defect,0
1,67,1,asymptomatic,160.0,286.0,0.0,lv hypertrophy,108.0,1.0,1.5,flat,3.0,normal,2
2,67,1,asymptomatic,120.0,229.0,0.0,lv hypertrophy,129.0,1.0,2.6,flat,2.0,reversable defect,1
3,37,1,non-anginal,130.0,250.0,0.0,normal,187.0,0.0,3.5,downsloping,0.0,normal,0
4,41,0,atypical angina,130.0,204.0,0.0,lv hypertrophy,172.0,0.0,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54,0,asymptomatic,127.0,333.0,1.0,st-t abnormality,154.0,0.0,0.0,NaN,NaN,NaN,1
916,62,1,typical angina,NaN,139.0,0.0,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,55,1,asymptomatic,122.0,223.0,1.0,st-t abnormality,100.0,0.0,0.0,NaN,NaN,fixed defect,2
918,58,1,asymptomatic,NaN,385.0,1.0,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


## Limpiamos las columnas categóricas

Usamos get_dummies para generar columnas con datos TRUE/FALSE de cada uno de los datos en las variables categóricas


In [6]:
# Variables categóricas que vamos a convertir
columnas_categoricas = ['sexo', 'tipo_dolor_pecho', 'ecg_reposo', 'prueba_talio', 'pendiente_ST']

# Aplicar One-Hot Encoding
df_limpio = pd.get_dummies(df, columns=columnas_categoricas, drop_first=False) # ponemos false ya que no importa la multicolinealidad. 
'''
en modelos lineales tener varias columnas que afectan totalmente a otras genera multicolinealidad, ejemplo:
si tenemos la columna sexo y hacemos un .getdummies() drop_first=True, solo nos genera sexo_mujer y el valor es 0 o 1, si es drop_first=False, genera
sexo_mujer 0 o 1 y sexo_hombre 0 o 1.
en regresión lineal y logística esa duplicidad puede afectar a los resultados, en knn, random forest y otros no afecta.
'''
# Verificamos que todo sea numérico o booleano
print(df_limpio.dtypes)
df_limpio.head()


edad                                  int64
presion_arterial                    float64
colesterol                          float64
glucosa_ayuno_alta                  float64
frecuencia_cardiaca_max             float64
angina_por_ejercicio                float64
depresion_ST                        float64
vasos_coloreados                    float64
enfermedad_cardiaca                   int64
sexo_0                                 bool
sexo_1                                 bool
tipo_dolor_pecho_asymptomatic          bool
tipo_dolor_pecho_atypical angina       bool
tipo_dolor_pecho_non-anginal           bool
tipo_dolor_pecho_typical angina        bool
ecg_reposo_lv hypertrophy              bool
ecg_reposo_normal                      bool
ecg_reposo_st-t abnormality            bool
prueba_talio_fixed defect              bool
prueba_talio_normal                    bool
prueba_talio_reversable defect         bool
pendiente_ST_downsloping               bool
pendiente_ST_flat               

,edad,presion_arterial,colesterol,glucosa_ayuno_alta,frecuencia_cardiaca_max,angina_por_ejercicio,depresion_ST,vasos_coloreados,enfermedad_cardiaca,sexo_0,...,tipo_dolor_pecho_typical angina,ecg_reposo_lv hypertrophy,ecg_reposo_normal,ecg_reposo_st-t abnormality,prueba_talio_fixed defect,prueba_talio_normal,prueba_talio_reversable defect,pendiente_ST_downsloping,pendiente_ST_flat,pendiente_ST_upsloping
0,63,145.0,233.0,1.0,150.0,0.0,2.3,0.0,0,False,...,True,True,False,False,True,False,False,True,False,False
1,67,160.0,286.0,0.0,108.0,1.0,1.5,3.0,2,False,...,False,True,False,False,False,True,False,False,True,False
2,67,120.0,229.0,0.0,129.0,1.0,2.6,2.0,1,False,...,False,True,False,False,False,False,True,False,True,False
3,37,130.0,250.0,0.0,187.0,0.0,3.5,0.0,0,False,...,False,False,True,False,False,True,False,True,False,False
4,41,130.0,204.0,0.0,172.0,0.0,1.4,0.0,0,True,...,False,True,False,False,False,True,False,False,False,True


## Cambiamos los balores booleanos a 1/0
-   como tenemos algunos NaN, los tenemos que convertir a .astype('Int64') para que esa columna los soporte

In [7]:
# Identificar las columnas booleanas
columnas_booleanas = df_limpio.select_dtypes(include=['bool']).columns

# Convertir solo las columnas booleanas a 'Int64' (permite NaN y enteros)
df_limpio[columnas_booleanas] = df_limpio[columnas_booleanas].astype('Int64')

# Verificar que todas las columnas sean numéricas y que los NaN no se han perdido
print(df_limpio.dtypes)
df_limpio.head()


edad                                  int64
presion_arterial                    float64
colesterol                          float64
glucosa_ayuno_alta                  float64
frecuencia_cardiaca_max             float64
angina_por_ejercicio                float64
depresion_ST                        float64
vasos_coloreados                    float64
enfermedad_cardiaca                   int64
sexo_0                                Int64
sexo_1                                Int64
tipo_dolor_pecho_asymptomatic         Int64
tipo_dolor_pecho_atypical angina      Int64
tipo_dolor_pecho_non-anginal          Int64
tipo_dolor_pecho_typical angina       Int64
ecg_reposo_lv hypertrophy             Int64
ecg_reposo_normal                     Int64
ecg_reposo_st-t abnormality           Int64
prueba_talio_fixed defect             Int64
prueba_talio_normal                   Int64
prueba_talio_reversable defect        Int64
pendiente_ST_downsloping              Int64
pendiente_ST_flat               

,edad,presion_arterial,colesterol,glucosa_ayuno_alta,frecuencia_cardiaca_max,angina_por_ejercicio,depresion_ST,vasos_coloreados,enfermedad_cardiaca,sexo_0,...,tipo_dolor_pecho_typical angina,ecg_reposo_lv hypertrophy,ecg_reposo_normal,ecg_reposo_st-t abnormality,prueba_talio_fixed defect,prueba_talio_normal,prueba_talio_reversable defect,pendiente_ST_downsloping,pendiente_ST_flat,pendiente_ST_upsloping
0,63,145.0,233.0,1.0,150.0,0.0,2.3,0.0,0,0,...,1,1,0,0,1,0,0,1,0,0
1,67,160.0,286.0,0.0,108.0,1.0,1.5,3.0,2,0,...,0,1,0,0,0,1,0,0,1,0
2,67,120.0,229.0,0.0,129.0,1.0,2.6,2.0,1,0,...,0,1,0,0,0,0,1,0,1,0
3,37,130.0,250.0,0.0,187.0,0.0,3.5,0.0,0,0,...,0,0,1,0,0,1,0,1,0,0
4,41,130.0,204.0,0.0,172.0,0.0,1.4,0.0,0,1,...,0,1,0,0,0,1,0,0,0,1


In [8]:
df_limpio.isna().sum()

edad                                  0
presion_arterial                     59
colesterol                           30
glucosa_ayuno_alta                   90
frecuencia_cardiaca_max              55
angina_por_ejercicio                 55
depresion_ST                         62
vasos_coloreados                    611
enfermedad_cardiaca                   0
sexo_0                                0
sexo_1                                0
tipo_dolor_pecho_asymptomatic         0
tipo_dolor_pecho_atypical angina      0
tipo_dolor_pecho_non-anginal          0
tipo_dolor_pecho_typical angina       0
ecg_reposo_lv hypertrophy             0
ecg_reposo_normal                     0
ecg_reposo_st-t abnormality           0
prueba_talio_fixed defect             0
prueba_talio_normal                   0
prueba_talio_reversable defect        0
pendiente_ST_downsloping              0
pendiente_ST_flat                     0
pendiente_ST_upsloping                0
dtype: int64

In [9]:
df_limpio = df_limpio.fillna(df_limpio.median())
df_limpio.isna().sum()


edad                                0
presion_arterial                    0
colesterol                          0
glucosa_ayuno_alta                  0
frecuencia_cardiaca_max             0
angina_por_ejercicio                0
depresion_ST                        0
vasos_coloreados                    0
enfermedad_cardiaca                 0
sexo_0                              0
sexo_1                              0
tipo_dolor_pecho_asymptomatic       0
tipo_dolor_pecho_atypical angina    0
tipo_dolor_pecho_non-anginal        0
tipo_dolor_pecho_typical angina     0
ecg_reposo_lv hypertrophy           0
ecg_reposo_normal                   0
ecg_reposo_st-t abnormality         0
prueba_talio_fixed defect           0
prueba_talio_normal                 0
prueba_talio_reversable defect      0
pendiente_ST_downsloping            0
pendiente_ST_flat                   0
pendiente_ST_upsloping              0
dtype: int64

## Dividimos los datos entre predictoras y objetivos

In [10]:
# Separar variables predictoras (X) y variable objetivo (y)
X = df_limpio.drop(columns=['enfermedad_cardiaca'])  # Eliminamos la columna objetivo
y = df_limpio['enfermedad_cardiaca']  # Variable objetivo



## Normalizamos los Datos
kNN usa distancias, así que si los datos tienen escalas diferentes, el modelo se sesga. Usamos StandardScaler para normalizar:

In [11]:
from sklearn.preprocessing import StandardScaler

# Normalizar los datos (para que kNN no dé más peso a valores grandes)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[ 1.00738556,  0.70517639,  0.30364317, ...,  3.68824818,
        -0.77459667, -0.5320941 ],
       [ 1.43203377,  1.51856943,  0.78996695, ..., -0.27113143,
         1.29099445, -0.5320941 ],
       [ 1.43203377, -0.65047866,  0.26693949, ..., -0.27113143,
         1.29099445, -0.5320941 ],
       ...,
       [ 0.15808914, -0.54202626,  0.21188397, ..., -0.27113143,
        -0.77459667, -0.5320941 ],
       [ 0.4765753 , -0.10821664,  1.69838307, ..., -0.27113143,
        -0.77459667, -0.5320941 ],
       [ 0.90122351, -0.65047866,  0.4963375 , ..., -0.27113143,
        -0.77459667, -0.5320941 ]], shape=(920, 23))

Manejamos Valores NaN
Si aún hay valores NaN, los rellenamos con un imputador basado en kNN: KNNImputer. Este lo que hace es aplicar knn a los X vecinos y remplaza los NaN con la media de los X vecinos. 

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Dividir en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Entrenar kNN con distancia adaptativa
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_train, y_train)

# Evaluar la precisión del modelo
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"🔹 Precisión del modelo kNN: {accuracy * 100:.2f}%")


🔹 Precisión del modelo kNN: 55.43%
